#### Notebook looks at the inner-workings of the Teacheable agent, and understand how it solves the problem of retaining its teachings

Teachable Agent required ChromaDB, which requires a long list of dependencies. So moving the notebook to Linux environment, and explore from there. 

Autogen with teachable module has to be installed
    pip install pyautogen[teachable]

As this notebook requires the linux environment, it will test the AssistantAgent execution with local "Codellama2" or "llama2" models. Will be updated later

Steps Followed:

- Setting up the Gradio endpoints in Kali Server, and test the generation end point in this notebook. (After further reviewing, it started working.)

- Create the config_list to only work on the Teachable agent and understand it

In [9]:
import autogen
import os
from dotenv import load_dotenv

codellama_config = autogen.config_list_from_dotenv(
    dotenv_file_path="../.env",
    model_api_key_map={
        "gpt-3.5-turbo":"OPENAI_API_KEY",
        "codellama":{
            "api_key_env_var": "HF_KEY",
            "api_version": "v0.2",
            "base_url":"http://127.0.0.1:7860",
        },
    },
    filter_dict={
        "model":{
            "codellama"
        }
    }
)

In [12]:
codellama_config

[{'api_key': 'sk-00000',
  'base_url': 'http://127.0.0.1:7860',
  'api_version': 'v0.2',
  'model': 'codellama'}]

In [19]:
config_list=[
{
"model": "codellama",
"base_url": "http://127.0.0.1:7860",
"api_type": "open_ai",
"api_key": "NULL", # just a placeholder
}
]

### following is deprecated
from autogen import oai

response = oai.ChatCompletion.create(
    config_list=[
        {
            "model": "CodeLlama-7b-hf",
            "api_base": "http://127.0.0.1:7860",
            "api_type": "open_ai",
            "api_key": "NULL",
        }
    ],
    messages=[{"role": "user", "content": "Hi"}]
)
print(response)

In [31]:
from autogen import OpenAIWrapper

client = OpenAIWrapper(
    config_list=[
       {
            "model": "CodeLlama2-7B-hf",
            "base_url": "http://127.0.0.1:7860/predict",
        }
    ],
)

In [32]:
client.create(messages=[{"role": "user", "content": "Python learning tips."}], model="CodeLlama2-7B-hf")
client.print_usage_summary()  # Display usage
client.clear_usage_summary()  

NotFoundError: Error code: 404 - {'detail': 'Not Found'}

In [27]:
client = OpenAIWrapper(
    config_list=[
       {
            "model": "gpt-3.5-turbo",
            "api_key": os.environ.get("OPENAI_API_KEY"),
            "base_url": "https://api.openai.com/v1",
        },
    ],
)

In [29]:
client.create(messages=[{"role": "user", "content": "Python learning tips."}], model="gpt-3.5-turbo")
client.print_usage_summary()  # Display usage
client.clear_usage_summary()  

----------------------------------------------------------------------------------------------------
No actual cost incurred (all completions are using cache).

Usage summary including cached usage: 
Total cost: 0.0009
* Model 'gpt-3.5-turbo-0613': cost: 0.0009, prompt_tokens: 11, completion_tokens: 440, total_tokens: 451
----------------------------------------------------------------------------------------------------


In [30]:
client.create(messages=[{"role": "user", "content": "Python learning tips."}], model="gpt-3.5-turbo")

ChatCompletion(id='chatcmpl-8VhcSZ9MMdJ1xu0fbPvdVJnDtsoWN', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="1. Start with the basics: Familiarize yourself with the syntax and structure of Python. Understand concepts like variables, data types, loops, conditionals, and functions.\n\n2. Code practice: Practice regularly by solving coding problems or challenges. Websites like HackerRank, LeetCode, and Codecademy offer a wide range of coding exercises.\n\n3. Work on small projects: Start with simple projects that align with your interests. It could be a calculator, a simple game, or a text-based program. Building projects helps you apply what you've learned and reinforces your understanding of Python.\n\n4. Read and explore documentation: Python has extensive documentation available for free. Learn to use the official Python documentation to understand different functions, libraries, and modules. Take advantage of online resources like tutorials, blogs

In [20]:
llama_assistant = autogen.AssistantAgent(
    name="code_assistant",
    llm_config={
        "cache_seed": 42,
        "config_list": config_list,
        "temperature": 0,
    },
    system_message="you are a top notch problem solve"
)

In [22]:
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    code_execution_config={
        "work_dir": "llama_code",
        "use_docker": False,
    }
)

In [23]:
user_proxy.initiate_chat(llama_assistant, message="What is the date today")

user_proxy (to code_assistant):

What is the date today

--------------------------------------------------------------------------------


TypeError: Completions.create() got an unexpected keyword argument 'api_type'